In [35]:
import os
os.chdir('../eval')
%load_ext autoreload

import hashlib
import time
import random
import json
import openai
import transformers
import fnmatch
import datasets
import pathlib
import torch

from functools import cache
from collections import Counter
from datasets import load_dataset
from warnings import warn
from abc import abstractmethod, ABC
from eval import tasks
from tasks import utils
from eval.utils import TokenizedDataset, complete_code
from eval.tasks.utils import evaluate, convert_to_nltk_rep
# from eval.generation import parallel_generations
from diskcache import Cache
from concurrent.futures import ThreadPoolExecutor


from torch.utils.data.dataloader import DataLoader
from transformers import StoppingCriteria, StoppingCriteriaList
from accelerate.utils import set_seed

from accelerate import Accelerator, DeepSpeedPlugin
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

from eval import tasks
from eval.generation import parallel_generations
from eval.args import RunnerArguments, HFArguments, OAIArguments, GenerationArguments
# from eval.evaluator import HFEvaluator, OAIEvaluator
from eval.tasks import ALL_TASKS, TASK_REGISTRY


transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FOL

In [48]:
premises = [ "∀x (RomanceLanguage(x) → IndoEuropeanLanguage(x))", "∀x (RomanceLanguage(x) → MemberOf(x, romancefamily))", "∀x ∀y ∀z ((MemberOf(x, z) ∧ MemberOf(y, z)) → (Related(x, y) ∧ Related(y, x)))", "RomanceLanguage(french) ∧ RomanceLanguage(spanish)", "Related(german, spanish)", "∀x (¬Related(basque, x))" ]
conclusion = 'IndoEuropeanLanguage(french)'

premises_prep = [utils.convert_to_nltk_rep(p) for p in premises]
conclusion_prep = utils.convert_to_nltk_rep(conclusion)

utils.evaluate(premises_prep, conclusion_prep)

'True'